In [0]:
from pyspark.sql.functions import sum,desc,avg,count,max,col,lag
from pyspark.sql.window import Window

<h3>dataset 1<\h3>

In [0]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("user_info").getOrCreate()


columns = ["user_id", "timestamp"]

data= [("user1", "2023-08-21 10:00:00"),
("user2", "2023-08-21 11:30:00"),
("user1", "2023-08-21 12:15:00"),
("user3", "2023-08-21 13:45:00"),
("user2", "2023-08-21 14:30:00"),
("user1", "2023-08-21 15:00:00")]

df=spark.createDataFrame(data,columns)
df.show()


+-------+-------------------+
|user_id|          timestamp|
+-------+-------------------+
|  user1|2023-08-21 10:00:00|
|  user2|2023-08-21 11:30:00|
|  user1|2023-08-21 12:15:00|
|  user3|2023-08-21 13:45:00|
|  user2|2023-08-21 14:30:00|
|  user1|2023-08-21 15:00:00|
+-------+-------------------+



<h3>𝐅𝐢𝐧𝐝 𝐭𝐡𝐞 𝐞𝐚𝐫𝐥𝐢𝐞𝐬𝐭 𝐚𝐧𝐝 𝐥𝐚𝐭𝐞𝐬𝐭 𝐭𝐢𝐦𝐞𝐬𝐭𝐚𝐦𝐩𝐬 𝐢𝐧 𝐭𝐡𝐞 𝐝𝐚𝐭𝐚𝐬𝐞𝐭.<\h3>

In [0]:
df.agg(max("timestamp").alias("latest_timestamp")).show()

+-------------------+
|   latest_timestamp|
+-------------------+
|2023-08-21 15:00:00|
+-------------------+



<h3>𝐂𝐨𝐮𝐧𝐭 𝐭𝐡𝐞 𝐧𝐮𝐦𝐛𝐞𝐫 𝐨𝐟 𝐚𝐜𝐭𝐢𝐯𝐢𝐭𝐢𝐞𝐬 𝐩𝐞𝐫 𝐮𝐬𝐞𝐫.<\h3>

In [0]:
df.groupBy("user_id").count().show()


+-------+-----+
|user_id|count|
+-------+-----+
|  user1|    3|
|  user2|    2|
|  user3|    1|
+-------+-----+



<h3>𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐡𝐞 𝐭𝐢𝐦𝐞 𝐝𝐮𝐫𝐚𝐭𝐢𝐨𝐧 𝐛𝐞𝐭𝐰𝐞𝐞𝐧 𝐜𝐨𝐧𝐬𝐞𝐜𝐮𝐭𝐢𝐯𝐞 𝐚𝐜𝐭𝐢𝐯𝐢𝐭𝐢𝐞𝐬 𝐟𝐨𝐫 𝐞𝐚𝐜𝐡 𝐮𝐬𝐞𝐫.</h3>

In [0]:
df = df.withColumn("timestamp", df["timestamp"].cast("timestamp"))
df1=df.withColumn("lag_time",lag("timestamp").over(Window.partitionBy("user_id").orderBy("timestamp")))
df1.withColumn("time_duration",col("timestamp").cast("long")-col("lag_time").cast("long")).show()

+-------+-------------------+-------------------+-------------+
|user_id|          timestamp|           lag_time|time_duration|
+-------+-------------------+-------------------+-------------+
|  user1|2023-08-21 10:00:00|               null|         null|
|  user1|2023-08-21 12:15:00|2023-08-21 10:00:00|         8100|
|  user1|2023-08-21 15:00:00|2023-08-21 12:15:00|         9900|
|  user2|2023-08-21 11:30:00|               null|         null|
|  user2|2023-08-21 14:30:00|2023-08-21 11:30:00|        10800|
|  user3|2023-08-21 13:45:00|               null|         null|
+-------+-------------------+-------------------+-------------+



<h3>Data set 2</h3>

In [0]:
data = [
    (1, "login", "2023-08-20 10:23:45"),
    (2, "view", "2023-08-20 11:15:30"),
    (1, "purchase", "2023-08-20 12:45:18"),
    (3, "view", "2023-08-20 13:30:22")
]
columns = ["user_id", "action", "timestamp"]

df=spark.createDataFrame(data,columns)
df.show()

+-------+--------+-------------------+
|user_id|  action|          timestamp|
+-------+--------+-------------------+
|      1|   login|2023-08-20 10:23:45|
|      2|    view|2023-08-20 11:15:30|
|      1|purchase|2023-08-20 12:45:18|
|      3|    view|2023-08-20 13:30:22|
+-------+--------+-------------------+



<h3>Count the number of actions.</h3>

In [0]:
df.groupBy("action").count().show()

+--------+-----+
|  action|count|
+--------+-----+
|   login|    1|
|    view|    2|
|purchase|    1|
+--------+-----+



<h3>What are the unique actions recorded in the dataset.</h3>

In [0]:
df.groupBy("action").agg(count("*").alias("cnt")).filter(col("cnt")==1).select("action").show()

+--------+
|  action|
+--------+
|   login|
|purchase|
+--------+



<h3>Calculate the time duration between consecutive activities for each user</h3>

In [0]:
df=df.withColumn("timestamp",df.timestamp.cast("timestamp"))
df1=df.withColumn("lag_time",lag("timestamp").over(Window.partitionBy("user_id").orderBy("timestamp")))
df1.withColumn("time_diff",df1.timestamp.cast("long")-df1.lag_time.cast("long")).show()

+-------+--------+-------------------+-------------------+---------+
|user_id|  action|          timestamp|           lag_time|time_diff|
+-------+--------+-------------------+-------------------+---------+
|      1|   login|2023-08-20 10:23:45|               null|     null|
|      1|purchase|2023-08-20 12:45:18|2023-08-20 10:23:45|     8493|
|      2|    view|2023-08-20 11:15:30|               null|     null|
|      3|    view|2023-08-20 13:30:22|               null|     null|
+-------+--------+-------------------+-------------------+---------+

